In [36]:
#conv Neural Network
# tensorboard --logdir=/home/ncc/notebook/learn/tensorboard/log
"""
created by kim Seong jung

"""
import numpy as np 
import tensorflow as tf
import re

import math
import time
import matplotlib.pyplot as plt
%matplotlib inline
import os 

file_locate='/home/user01/notebook/eye_numpy_64/'
sess = tf.InteractiveSession()
test_img=np.load(file_locate+'test_img.npy');
try:
    print np.shape(test_img)
    img_row = np.shape(test_img)[1]
    img_col = np.shape(test_img)[2]
except:
    np.shape(test_img)
    test_img=np.reshape(test_img , newshape = [np.shape(test_img)[0] , 32, 32 ,3] )
    img_row = np.shape(test_img)[1]
    img_col = np.shape(test_img)[2]

    
divide_flag= False
aug_flag = False
save_flag = False
restore_flag = False
batch_size=30
print img_row ,img_col
n_classes =2
in_ch =3
out_ch1=100
out_ch2=100
out_ch3=100
out_ch4=100
out_ch5=100


fully_ch1=1024
fully_ch2 =1024
fully_ch3 =1024



c_ksize1 =[11,11,in_ch , out_ch1]
c_ksize2 =[5,5, out_ch1, out_ch2]
c_ksize3 =[3,3, out_ch2, out_ch3]
c_ksize4 =[3,3, out_ch3, out_ch4]
c_ksize5 =[3,3, out_ch4, out_ch5]

c_strides1=[1,4,4,1]
c_strides2=[1,1,1,1]
c_strides3=[1,1,1,1]
c_strides4=[1,1,1,1]
c_strides5=[1,1,1,1]



p_ksize1=[1,3,3,1]
p_ksize2=[1,3,3,1]
p_ksize5=[1,3,3,1]

p_strides1=[1,2,2,1]
p_strides2=[1,2,2,1]
p_strides5=[1,2,2,1]

iterate=30000




print img_col , img_row

(2789, 128, 128, 3)
128 128
128 128


In [37]:
if aug_flag == False:
    x= tf.placeholder("float",shape=[None,img_row , img_col , in_ch],  name = 'x-input')
    y_=tf.placeholder("float",shape=[None , n_classes] , name = 'y-input')
    keep_prob = tf.placeholder("float")

    x_image= tf.reshape(x,[-1,img_row,img_col,3])

elif aug_flag == True:
    crop_img_col =118
    crop_img_row =118
    
    x= tf.placeholder("float",shape=[None,crop_img_row , crop_img_row , in_ch],  name = 'x-input')
    y_=tf.placeholder("float",shape=[None , n_classes] , name = 'y-input')
    keep_prob = tf.placeholder("float")

    x_image= tf.reshape(x,[-1,crop_img_row,crop_img_col,in_ch])


In [38]:
import os 
os.getcwd()

'/home/user01/notebook'

In [39]:
with tf.device('/gpu:3'):
#with tf.device('/gpu:1'):

    if divide_flag == False:
        train_img=np.load(file_locate+'train_img.npy');
        train_lab=np.load(file_locate+'train_lab.npy');
        val_img= np.load(file_locate+'val_img.npy');
        val_lab = np.load(file_locate+'val_lab.npy');
        test_img=np.load(file_locate+'test_img.npy');
        test_lab=np.load(file_locate+'test_lab.npy');

        print "Training Data",np.shape(train_img)
        print "Training Data Label",np.shape(train_lab)
        print "Test Data Label",np.shape(test_lab)
        print "val Data Label" , np.shape(val_img)

        n_train= np.shape(train_img)[0]
        n_train_lab = np.shape(train_lab)[0]

    if divide_flag == True:
        train_img=np.load(file_locate+'train_img_1.npy');
        train_lab=np.load(file_locate+'train_lab_1.npy');
        val_img= np.load(file_locate+'val_img.npy');
        val_lab = np.load(file_locate+'val_lab.npy');
        test_img=np.load(file_locate+'test_img.npy');
        test_lab=np.load(file_locate+'test_lab.npy');

        print "Training Data",np.shape(train_img)
        print "Training Data Label",np.shape(train_lab)
        print "Test Data Label",np.shape(test_lab)
        print "val Data Label" , np.shape(val_lab)

        n_train= np.shape(train_img)[0]
        n_train_lab = np.shape(train_lab)[0]


Training Data (22302, 128, 128, 3)
Training Data Label (22302, 2)
Test Data Label (2789, 2)
val Data Label (2787, 128, 128, 3)


In [40]:
"""def weight_variable(name,shape):
    #initial = tf.truncated_normal(shape , stddev=0.1)
    initial = tf.get_variable(name,shape=shape , initializer = tf.contrib.layers.xavier_initializer())
    return tf.Variable(initial)"""

def bias_variable(shape):
    initial = tf.constant(0.1 , shape=shape)
    return tf.Variable(initial)



In [41]:
with tf.device('/gpu:0'):
    def next_batch(batch_size , image , label):

        a=np.random.randint(np.shape(image)[0] -batch_size)
        batch_x = image[a:a+batch_size,:]
        batch_y= label[a:a+batch_size,:]
        return batch_x, batch_y


In [42]:


def conv2d(x,w,strides_):
    return tf.nn.conv2d(x,w, strides = strides_, padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x , ksize=[1,2,2,1] ,strides = [1,2,2,1] , padding = 'SAME')
def max_pool(x , ksize , strides , padding='SAME'):
    return tf.nn.max_pool(x ,ksize , strides , padding )
def make_weights_biases(layer_name , w_name , ksize ,device_name,initializer='xavier'):
    if len(ksize)==4: # convolution filter shape [batch , row , col , color_ch]
        out_ch=ksize[3]
    elif len(ksize)==2: #fully connected layer shape [in_ch , output_ch]
        out_ch=ksize[1]
    with tf.device(device_name):
        with tf.variable_scope(layer_name) as scope:
            try:
                w_conv = tf.get_variable(w_name, ksize , initializer = tf.contrib.layers.xavier_initializer())
            except:
                scope.reuse_variables()
                w_conv = tf.get_variable(w_name, ksize , initializer = tf.contrib.layers.xavier_initializer())
        
        with tf.variable_scope(layer_name) as scope:
            try:
                b_conv = bias_variable([out_ch])
            except:
                scope.reuse_variables()
                b_conv = bias_variable([out_ch])
    return w_conv , b_conv 




# make weights and biases that is used to convolution layer 

In [43]:
w_conv1 , b_conv1 =make_weights_biases('layer1' , 'W1' , c_ksize1 ,device_name = '/gpu:2')
w_conv2 , b_conv2= make_weights_biases('layer2' , 'W2' , c_ksize2 ,device_name = '/gpu:2')
w_conv3 , b_conv3= make_weights_biases('layer3' , 'W3' , c_ksize3 ,device_name = '/gpu:2')
w_conv4 , b_conv4= make_weights_biases('layer4' , 'W4' , c_ksize4 ,device_name = '/gpu:2')
w_conv5 , b_conv5= make_weights_biases('layer5' , 'W5' , c_ksize5 ,device_name = '/gpu:2')

print w_conv1.get_shape()
print w_conv2.get_shape()
print w_conv3.get_shape()
print w_conv4.get_shape()
print w_conv5.get_shape()

(11, 11, 3, 100)
(5, 5, 100, 100)
(3, 3, 100, 100)
(3, 3, 100, 100)
(3, 3, 100, 100)


# connect Convolution layer

In [44]:

print x_image
#conncect hidden layer 
with tf.device('/gpu:4'):
    h_conv1 = tf.nn.relu(conv2d(x_image , w_conv1 ,c_strides1)+b_conv1)
    h_conv1 = max_pool(h_conv1 , p_ksize1 , p_strides1)
    print h_conv1
    
    #print conv2d(h_pool1 , w_conv2).get_shape()


Tensor("Reshape_5:0", shape=(?, 128, 128, 3), dtype=float32)
Tensor("MaxPool_3:0", shape=(?, 16, 16, 100), dtype=float32, device=/device:GPU:4)


In [45]:
with tf.device('/gpu:4'):
    h_conv2 = tf.nn.relu(conv2d(h_conv1 , w_conv2 ,c_strides2)+b_conv2)
    h_conv2 = max_pool(h_conv2 , p_ksize2 , p_strides2)#pooling   
    print h_conv2

Tensor("MaxPool_4:0", shape=(?, 8, 8, 100), dtype=float32, device=/device:GPU:4)


In [46]:
with tf.device('/gpu:4'):
    h_conv3 = tf.nn.relu(conv2d(h_conv2 , w_conv3,c_strides3)+b_conv3)
    print h_conv3

Tensor("Relu_7:0", shape=(?, 8, 8, 100), dtype=float32, device=/device:GPU:4)


In [47]:
with tf.device('/gpu:4'):
    h_conv4 = tf.nn.relu(conv2d(h_conv3 , w_conv4,c_strides4)+b_conv4)
    print h_conv4

Tensor("Relu_8:0", shape=(?, 8, 8, 100), dtype=float32, device=/device:GPU:4)


In [48]:
with tf.device('/gpu:4'):
    h_conv5 = tf.nn.relu(conv2d(h_conv4, w_conv5,c_strides5)+b_conv5)
    h_conv5= max_pool(h_conv5 , p_ksize5 , p_strides5) #pooling 
    print h_conv5

Tensor("MaxPool_5:0", shape=(?, 4, 4, 100), dtype=float32, device=/device:GPU:4)


In [49]:
end_conv = h_conv5


In [50]:

end_conv_row=int(h_conv5.get_shape()[1])
end_conv_col=int(h_conv5.get_shape()[2])
end_conv_ch=int(h_conv5.get_shape()[3])
#connect fully connected layer 

# flat convolution layer 

In [51]:
with tf.device('/gpu:2'): # flat conv layer 
    end_flat_conv =tf.reshape(end_conv, [-1,end_conv_col*end_conv_row*end_conv_ch])

# make weights and biases that used fully connected layer 


In [52]:
fc_ksize1=[end_conv_col*end_conv_row*end_conv_ch,fully_ch1]
fc_ksize2=[fully_ch1,fully_ch2]
   

In [53]:
w_fc1 ,b_fc1 = make_weights_biases('fc1' , 'fc_W1' , fc_ksize1 ,  '/gpu:3')
w_fc2 ,b_fc2 = make_weights_biases('fc2' , 'fc_W2' , fc_ksize2 ,  '/gpu:3')


In [54]:
h_fc1=tf.matmul(end_flat_conv , w_fc1 )+b_fc1
h_fc1=tf.nn.dropout(h_fc1 , keep_prob)

In [55]:
h_fc2=tf.matmul(h_fc1 , w_fc2 )+b_fc2
h_fc2=tf.nn.dropout(h_fc2 , keep_prob)

In [56]:
end_fc=h_fc2


In [57]:

end_ksize=[end_fc.get_shape()[1] , n_classes]   
w_end ,b_end = make_weights_biases('fc_end' , 'fc_end_W' , end_ksize ,  '/gpu:3')
y_conv = tf.matmul(end_fc , w_end)+b_end
    

In [58]:
#dirname = '/home/ncc/notebook/mammo/result/'

dirname='/home/user01/notebook/'
    
count=0
while(True):
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
        break
    elif not os.path.isdir(dirname + str(count)):
        dirname=dirname+str(count)
        os.mkdir(dirname)
        break
    else:
        count+=1
print 'it is recorded at :'+str(count)

it is recorded at :81


In [59]:
f=open(dirname+"/log.txt",'w')

In [60]:
def get_batch_list(folder_path):
    list_files=os.walk(folder_path).next()[2]
    print list_files
    ret_train_img_list=[]
    ret_train_lab_list=[]
    for i , ele in enumerate(list_files):

        if 'train'  in ele and 'img'in ele:
            ret_train_img_list.append(ele)
        elif 'train' in ele  and  'lab' in ele:
            ret_train_lab_list.append(ele)
    return ret_train_img_list ,ret_train_lab_list 

In [61]:
train_images , train_labels  = get_batch_list(file_locate)

['test_lab.npy', 'val_lab.npy', 'train_img.npy', 'test_img.npy', 'val_img.npy', 'train_lab.npy']


In [62]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]


train_images.sort(key=natural_keys)
train_labels.sort(key = natural_keys)
print(train_images)
print(train_labels)

['train_img.npy']
['train_lab.npy']


In [63]:
def aug(np_img ,img_row ,img_col , color_ch, crop_img_row , crop_img_col , label):
    
    n_img = np.shape(np_img)[0] 
    n_ret_img = n_img*(img_row - crop_img_row) * (img_col - crop_img_col)*2
    ret_images = np.zeros([n_ret_img ,crop_img_row , crop_img_col,color_ch])
    len_label= np.shape(label)[1]
    ret_labels =  np.zeros([n_ret_img  ,len_label])
    #print "n_augmented image size : " , n_ret_img 
    #print "n classes :", len_label
    #copy label in factor by 2014
    count=0
    if len(np.shape(np_img))==2:
        np_img=np.reshape(np_img , newshape = [np.shape(np_img)[0] , img_row , img_col ,color_ch])
        print np.shape(np_img)
    for n  in range(len(np_img)):
        for r in range(img_row - crop_img_row):
            for c in range(img_col - crop_img_col):
                
                ret_labels[n*2 +count, : ] = label[n,:]
                ret_labels[n*2+1+count , : ] = label[n,:]
                cropped_img = np_img[n, r:crop_img_row +r , c:crop_img_col+c ,: ]                
                ret_images[n*2+count,:,:,:]=cropped_img  
                ret_images[(n*2+1)+count , :,:,:] =np.fliplr(cropped_img )
                count+=1
    
    return ret_images ,ret_labels



In [64]:
def extract_test_img(np_img ,img_row ,img_col , color_ch, crop_img_row , crop_img_col ):
    left_top =(0,0)
    right_top =(  img_row  - crop_img_row  , 0 )
    center =  ((img_row  - crop_img_row)/2  , (img_col - crop_img_row)/2)
    left_buttom = (0,(img_col - crop_img_row)/2 )
    right_buttom =  (img_row  - crop_img_row , img_col - crop_img_row)
    
    left_top_images  = np_img[: , left_top[0]:crop_img_row+left_top[0] , left_top[1] : crop_img_col+left_top[1] , :  ]
    right_top_images = np_img[: , right_top[0]:crop_img_row +right_top[0], right_top[1] : crop_img_col +right_top[1], :  ]
    center_images    = np_img[: , center[0]:crop_img_row +center[0], center[1] : crop_img_col +center[1], :  ]
    left_buttom_images=np_img[: , left_buttom[0]:crop_img_row +left_buttom[0], left_buttom[1] : crop_img_col +left_buttom[1], :  ]
    right_buttom_images= np_img[: , right_buttom[0]:crop_img_row+right_buttom[0] , right_buttom[1] : crop_img_col +right_buttom[1] , :  ]

    
        
    return left_top_images , right_top_images , center_images , left_buttom_images , right_buttom_images 



# save and restore Model

In [67]:
if save_flag == True:
    global_step = tf.Variable(0 , trainable = False ,name = 'global_step' )

    saver = tf.train.Saver()
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())
    save_path = './ckpt'

In [68]:
if restore_flag == True:
    ckpt = tf.train.get_checkpoint_state(save_path)
    if ckpt and ckpt.model_checkpoint_path:
        print ckpt.model_checkpoint_path
        saver.restore(sess , ckpt.model_checkpoint_path)

In [69]:
if save_flag == True:
    a=sess.run(global_step)
    print a

In [33]:
with tf.device('/gpu:3'):
#sm_conv= tf.nn.softmax(y_conv)
    #cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
    start_time = time.time()

    regular=0.01*(tf.reduce_sum(tf.square(y_conv)))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( y_conv, y_))
with tf.device('/gpu:4'):
    cost = cost+regular
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cost) #1e-4
    with tf.name_scope("accuracy"):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y_conv,1) ,tf.argmax(y_,1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction , "float")) 



In [35]:



if divide_flag ==True:
    n_batch =len(train_images)
    batch_count=0

for i in range(iterate): 
    #print i
    if divide_flag ==True:
        print batch_count , i
        if batch_count >= n_batch:    
            train_img =np.load(file_locate+train_images[batch_count])
            train_lab =np.load(file_locate+train_labels[batch_count])
        
    batch_xs , batch_ys = next_batch(batch_size, train_img , train_lab)    
    if i%100 ==0: # in here add to validation 
        try:
            if aug_flag == True:
                color_ch = in_ch
                val_images=extract_test_img(val_img ,128 , 128 , color_ch   ,crop_img_row =118 , crop_img_col =118 )
                val_acc_list=[]
                val_loss_list=[]
                for img_ind ,ele  in enumerate(val_images): 
                    val_accuracy = sess.run( accuracy , feed_dict={x:ele , y_:val_lab , keep_prob: 1.0})        
                    val_loss = sess.run(cost , feed_dict = {x:ele , y_: val_lab , keep_prob: 1.0})

                    val_acc_list.append(val_accuracy)
                    val_loss_list.append(val_loss)

                val_acc_list=np.asarray(val_acc_list)
                val_loss_list=np.asarray(val_loss_list)
                val_acc=np.mean(val_acc_list)
                val_loss=np.mean(val_loss_list)


                train_images=extract_test_img(val_img ,128 , 128 , color_ch   ,crop_img_row =118 , crop_img_col =118 )
                train_acc_list=[]
                train_loss_list=[]
                for img_ind ,ele  in enumerate(train_images): 
                    train_accuracy = sess.run( accuracy , feed_dict={x:ele , y_:batch_ys , keep_prob: 1.0})        
                    train_loss = sess.run(cost , feed_dict = {x:ele , y_: batch_ys , keep_prob: 1.0})

                    train_acc_list.append(train_accuracy)
                    train_loss_list.append(train_loss)

                train_acc_list=np.asarray(train_acc_list)
                train_loss_list=np.asarray(train_loss_list)
                train_acc=np.mean(train_acc_list)
                train_loss=np.mean(train_loss_list)



            else:
                val_accuracy = sess.run( accuracy , feed_dict={x:val_img , y_:val_lab , keep_prob: 1.0})        
                val_loss = sess.run(cost , feed_dict = {x:val_img , y_: val_lab , keep_prob: 1.0})

                train_accuracy = sess.run( accuracy , feed_dict={x:batch_xs , y_:batch_ys , keep_prob: 1.0})        
                train_loss = sess.run(cost , feed_dict = {x:batch_xs, y_: batch_ys, keep_prob: 1.0})

            #result = sess.run(sm_conv , feed_dict = {x:val_img , y_:batch_ys , keep_prob :1.0})
            print i
            print("step %d , training  accuracy %g" %(i,train_accuracy))
            print("step %d , loss : %g" %(i,train_loss))
            train_str = 'step:\t'+str(i)+'\tval_loss:\t'+str(train_loss) +'\tval accuracy:\t'+str(train_accuracy)+'\n'

            print("step %d , validation  accuracy %g" %(i,val_accuracy))
            print("step %d , validation loss : %g" %(i,val_loss))
            val_str = 'step:\t'+str(i)+'\tval_loss:\t'+str(val_loss) +'\tval accuracy:\t'+str(val_accuracy)+'\n'


            f.write(val_str)
            f.write(train_str)
            if divide_flag ==True:
                batch_count+=1
        except :
            val_acc_list=[]
            val_loss_list=[]
            train_acc_list=[]
            train_loss_list=[]
            n_divide=len(val_img)/batch_size
            j=0
            if aug_flag == True:
                for j in range(n_divide):

                    # j*batch_size :(j+1)*batch_size
                    val_batch_xs =val_img[ j*batch_size :(j+1)*batch_size] 
                    val_batch_ys=val_lab[ j*batch_size :(j+1)*batch_size]
                    val_images = extract_test_img( val_batch_xs  ,128 , 128 , color_ch   ,crop_img_row =118 , crop_img_col =118 )
                    
                    for img_ind, ele in enumerate(val_images):                        
                        val_accuracy,val_loss = sess.run([accuracy ,cost], feed_dict={x:ele , y_:val_batch_ys , keep_prob: 1.0})        
                        val_acc_list.append(float(val_accuracy))
                        val_loss_list.append(float(val_loss))
                    #right above code have to modify
                val_batch_xs = val_img[ (j+1)*batch_size :  ] 
                val_batch_ys = val_lab[ (j+1)*batch_size :  ]
                val_images = extract_test_img( val_batch_xs  ,128 , 128 , color_ch   ,crop_img_row =118 , crop_img_col =118 )
                for img_ind , ele in enumerate(val_images):                        
                    val_accuracy,val_loss = sess.run([accuracy ,cost], feed_dict={x:ele , y_:val_batch_ys , keep_prob: 1.0})        
                    val_acc_list.append(float(val_accuracy))
                    val_loss_list.append(float(val_loss))
                
                val_acc_list=np.asarray(val_acc_list)
                val_loss_list= np.asarray(val_loss_list)

                val_acc=np.mean(val_acc_list)
                val_loss = np.mean(val_loss_list)

                #result = sess.run(sm_conv , feed_dict = {x:val_img , y_:batch_ys , keep_prob :1.0})
                train_images = extract_test_img( batch_xs  ,128 , 128 , color_ch   ,crop_img_row =118 , crop_img_col =118 )
                for img_ind, ele in enumerate(train_images):
                    train_accuracy,train_loss = sess.run([accuracy ,cost], feed_dict={x:ele , y_:batch_ys , keep_prob: 1.0})        
                    train_acc_list.append(float(train_accuracy))
                    train_loss_list.append(float(train_loss))
                train_acc_list=np.asarray(train_acc_list) 
                train_loss_list= np.asarray(train_loss_list)
                train_accuracy=np.mean(train_acc_list)
                train_loss=np.mean(train_loss_list)
            else:
                
                for j in range(n_divide):
                    # j*batch_size :(j+1)*batch_size
                    val_batch_xs =  val_img[ j*batch_size :(j+1)*batch_size] 
                    val_batch_ys =  val_lab[ j*batch_size :(j+1)*batch_size]                        
                    val_accuracy,val_loss = sess.run([accuracy ,cost], feed_dict={x:val_batch_xs , y_:val_batch_ys , keep_prob: 1.0})        
                    list_acc.append(float(val_accuracy))
                    list_loss.append(float(val_loss))
                    #right above code have to modify
                val_batch_xs = val_img[ (j+1)*batch_size :  ] 
                val_batch_ys = val_lab[ (j+1)*batch_size :  ]
                list_acc=np.asarray(list_acc)
                list_loss= np.asarray(list_loss)
                val_accuracy=np.mean(list_acc)
                val_loss = np.mean(list_loss)

                #result = sess.run(sm_conv , feed_dict = {x:val_img , y_:batch_ys , keep_prob :1.0})

                train_accuracy = sess.run( accuracy , feed_dict={x:batch_xs , y_:batch_ys , keep_prob: 1.0})        
                train_loss = sess.run(cost , feed_dict = {x:batch_xs, y_: batch_ys, keep_prob: 1.0})
                                      
                                      
                                      
            print("step %d , training  accuracy %g" %(i,train_accuracy))
            print("step %d , loss : %g" %(i,train_loss))
            train_str = 'step:\t'+str(i)+'\tval_loss:\t'+str(train_loss) +'\tval accuracy:\t'+str(train_accuracy)+'\n'

            print("step %d , validation  accuracy %g" %(i,val_accuracy))
            print("step %d , validation loss : %g" %(i,val_loss))
            val_str = 'step:\t'+str(i)+'\tval_loss:\t'+str(val_loss) +'\tval accuracy:\t'+str(val_accuracy)+'\n'

            f.write(val_str)
            f.write(train_str)
            if divide_flag == True:
                batch_count+=1
        if save_flag == True:
            global_ind=global_step.assign(i).eval()
            saver.save(sess , save_path + "/model.ckpt", global_step = global_ind)
        print 'model saved'
    if aug_flag == True:
        aug_batch_xs , aug_batch_ys=aug(np_img=batch_xs[:int(batch_size)] ,img_row= img_row  ,img_col =img_col\
                                        , color_ch=3, crop_img_row =118, crop_img_col =118, label = batch_ys )
        
        #after augmentation process ,
        #print 'a'
        share= len(aug_batch_xs) / batch_size
        #print share
        for i in xrange(share):
            #print i
            batch_xs=aug_batch_xs[i*batch_size : (i+1)*batch_size,:,:,:]  
            batch_ys=aug_batch_ys[i*batch_size : (i+1)*batch_size,:]  
            sess.run(train_step ,feed_dict={x: batch_xs, y_:batch_ys , keep_prob : 0.7})
            #print 'training'
        #print (i+1)*batch_size 
        #batch_xs=aug_batch_xs[(i+1)*batch_size :,:,:,:]  
        #batch_ys = aug_batch_ys[(i+1)*batch_size:,:]  
        #sess.run(train_step ,feed_dict={x: batch_xs, y_:batch_ys , keep_prob : 0.7})
        #print 'last training'
        #print len(aug_batch_xs)
        share= len(aug_batch_xs) / batch_size

        
    else:
        sess.run(train_step ,feed_dict={x: batch_xs, y_:batch_ys , keep_prob : 0.7})
    
print("--- Training Time : %s ---" % (time.time() - start_time))
train_time="--- Training Time : ---:\t" +str(time.time() - start_time)
f.write(train_time)

0
step 0 , training  accuracy 0.466667
step 0 , loss : 42.4895
step 0 , validation  accuracy 0.494438
step 0 , validation loss : 3483.31
model saved


FailedPreconditionError: Attempting to use uninitialized value beta1_power
	 [[Node: beta1_power/read = Identity[T=DT_FLOAT, _class=["loc:@layer1/W1"], _device="/job:localhost/replica:0/task:0/gpu:2"](beta1_power)]]
	 [[Node: beta1_power/read/_261 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:3", send_device="/job:localhost/replica:0/task:0/gpu:2", send_device_incarnation=1, tensor_name="edge_88_beta1_power/read", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:3"]()]]

Caused by op u'beta1_power/read', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_i

In [ ]:
aug_batch_xs , aug_batch_ys=aug(np_img=batch_xs[:] ,img_row= img_row  ,img_col =img_col\
                                        , color_ch=3, crop_img_row =118, crop_img_col =118, label = batch_ys )
        

In [ ]:
a=103 
share=a/10
for i in range(share):
    print i

In [ ]:
for i in range(3):
    print ''
print i

In [ ]:
try:
    test_accuracy = sess.run( accuracy , feed_dict={x:test_img , y_:test_lab , keep_prob: 1.0})        
    test_loss = sess.run(cost , feed_dict = {x:test_img , y_: test_lab , keep_prob: 1.0})

    #result = sess.run(sm_conv , feed_dict = {x:test_img , y_:batch_ys , keep_prob :1.0})
    print("step %d , testidation  accuracy %g" %(i,test_accuracy))
    print("step %d , testidation loss : %g" %(i,test_loss))
    test_str = 'step:\t'+str(i)+'\ttest_loss:\t'+str(test_loss) +'\ttest accuracy:\t'+str(test_accuracy)+'\n'

    f.write(test_str)
except :
    list_acc=[]
    list_loss=[]
    n_divide=len(test_img)/batch_size
    for j in range(n_divide):

        # j*batch_size :(j+1)*batch_size
        test_accuracy,test_loss = sess.run([accuracy ,cost], feed_dict={x:test_img[ j*batch_size :(j+1)*batch_size] , y_:test_lab[ j*batch_size :(j+1)*batch_size ] , keep_prob: 1.0})        
        list_acc.append(float(test_accuracy))
        list_loss.append(float(test_loss))
    test_accuracy , test_loss=sess.run([accuracy,cost] , feed_dict={x:test_img[(j+1)*batch_size : ] , y_:test_lab[(j+1)*(batch_size) : ] , keep_prob : 1.0})
    #right above code have to modify

    list_acc.append(test_accuracy)
    list_loss.append(test_loss)
    list_acc=np.asarray(list_acc)
    list_loss= np.asarray(list_loss)

    test_accuracy=np.mean(list_acc)
    test_loss = np.mean(list_loss)

    #result = sess.run(sm_conv , feed_dict = {x:test_img , y_:batch_ys , keep_prob :1.0})
    print("step %d , testidation  accuracy %g" %(i,test_accuracy))
    print("step %d , testidation loss : %g" %(i,test_loss))
    test_str = 'step:\t'+str(i)+'\ttest_loss:\t'+str(test_loss) +'\ttest accuracy:\t'+str(test_accuracy)+'\n'

    f.write(test_str)


In [ ]:
sess.close()
